In [1]:
SIZE=179
MIN_COUNT=5
WINDOW=6
NEGATIVE_WORDS=6
EPOCH_SIZE=4

In [2]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# shuffle
from random import shuffle

# logging
import logging
import os.path
import sys

program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))

C:\ProgramData\Miniconda2\envs\keras\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-04-10 02:31:32,259 : INFO : running C:\ProgramData\Miniconda2\envs\keras\lib\site-packages\ipykernel_launcher.py -f C:\Users\perfe\AppData\Roaming\jupyter\runtime\kernel-b3998bc0-27f0-4433-b0aa-ca0b922c3dd1.json


In [3]:


class LabeledLineSentence(object):

    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(
                        utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences

    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences
sources = {'test-neg.txt':'TEST_NEG', 'test-pos.txt':'TEST_POS', 'train-neg.txt':'TRAIN_NEG', 'train-pos.txt':'TRAIN_POS'} #'train-unsup.txt':'TRAIN_UNS'}

sentences = LabeledLineSentence(sources)


model = Doc2Vec(min_count=MIN_COUNT, window=WINDOW, size=SIZE, sample=1e-4, negative=5, workers=7)

model.build_vocab(sentences.to_array())





2017-04-10 02:31:32,290 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-04-10 02:31:32,298 : INFO : collecting all words and their counts
2017-04-10 02:31:32,299 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-04-10 02:31:32,307 : INFO : collected 2385 word types and 454 unique tags from a corpus of 454 examples and 36524 words
2017-04-10 02:31:32,308 : INFO : Loading a fresh vocabulary
2017-04-10 02:31:32,313 : INFO : min_count=5 retains 848 unique words (35% of original 2385, drops 1537)
2017-04-10 02:31:32,314 : INFO : min_count=5 leaves 33541 word corpus (91% of original 36524, drops 2983)
2017-04-10 02:31:32,318 : INFO : deleting the raw counts dictionary of 2385 items
2017-04-10 02:31:32,320 : INFO : sample=0.0001 downsamples 552 most-common words
2017-04-10 02:31:32,321 : INFO : downsampling leaves estimated 9895 word corpus (29.5% of prior 33541)
2017-04-10 02:31:32,322 : INFO : estimated required memo

In [4]:
for epoch in range(EPOCH_SIZE):
    logger.info('Epoch %d' % epoch)
    model.train(sentences.sentences_perm())

2017-04-10 02:31:32,353 : INFO : Epoch 0
2017-04-10 02:31:32,355 : INFO : training model with 7 workers on 848 vocabulary and 179 features, using sg=0 hs=0 sample=0.0001 negative=5 window=6
2017-04-10 02:31:32,357 : INFO : expecting 454 sentences, matching count from corpus used for vocabulary survey
2017-04-10 02:31:32,467 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-04-10 02:31:32,469 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-04-10 02:31:32,484 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-04-10 02:31:32,491 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-04-10 02:31:32,498 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-04-10 02:31:32,500 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-10 02:31:32,501 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-10 02:31:32,502 : INFO : training on 182620 raw words

In [5]:
model.save('./problems.d2v')


2017-04-10 02:31:32,933 : INFO : saving Doc2Vec object under ./problems.d2v, separately None
2017-04-10 02:31:32,935 : INFO : not storing attribute syn0norm
2017-04-10 02:31:32,935 : INFO : not storing attribute cum_table
2017-04-10 02:31:32,955 : INFO : saved ./problems.d2v


In [6]:
model = Doc2Vec.load('./problems.d2v')

2017-04-10 02:31:32,984 : INFO : loading Doc2Vec object from ./problems.d2v
2017-04-10 02:31:33,001 : INFO : loading wv recursively from ./problems.d2v.wv.* with mmap=None
2017-04-10 02:31:33,001 : INFO : setting ignored attribute syn0norm to None
2017-04-10 02:31:33,002 : INFO : loading docvecs recursively from ./problems.d2v.docvecs.* with mmap=None
2017-04-10 02:31:33,003 : INFO : setting ignored attribute cum_table to None
2017-04-10 02:31:33,005 : INFO : loaded ./problems.d2v


In [7]:
model.most_similar('tree')

2017-04-10 02:31:33,015 : INFO : precomputing L2-norms of word weight vectors


[('binary', 0.9998446106910706),
 ('its', 0.9998146295547485),
 ('traversal', 0.9998093843460083),
 ('values', 0.999808132648468),
 ('nodes', 0.9997954964637756),
 ('null', 0.9997896552085876),
 ('wikipedia', 0.999787449836731),
 ('like', 0.9997801780700684),
 ("node's", 0.9997782707214355),
 ('node', 0.9997777938842773)]

In [8]:
model.docvecs['TRAIN_POS_0']

array([  5.97662851e-02,   4.28803451e-02,   4.70422618e-02,
        -5.40076233e-02,   6.68402063e-03,  -1.50082648e-01,
         2.74657793e-02,   2.03467999e-02,  -1.22520123e-02,
        -9.92507190e-02,   8.51491764e-02,  -3.82563509e-02,
        -1.08294442e-01,  -2.27265470e-02,   5.13738915e-02,
         1.18339937e-02,  -1.10091399e-02,   5.99853136e-02,
         3.93416509e-02,  -6.67271987e-02,  -4.21812013e-03,
        -3.80047373e-02,  -1.00590758e-01,  -7.55948648e-02,
         1.05049193e-01,   9.52897221e-03,   5.21955155e-02,
        -2.55325623e-02,   8.40167403e-02,  -1.53658139e-02,
         4.11241502e-02,   3.58989537e-02,   7.76713490e-02,
        -1.06333114e-01,  -1.67350844e-02,   3.32841463e-02,
        -1.57632935e-03,   3.42021286e-02,  -2.16884781e-02,
        -5.39727248e-02,  -6.01158552e-02,   3.98898534e-02,
         9.34761539e-02,  -1.29023809e-02,   1.35793565e-02,
        -3.50974873e-02,  -4.00638133e-02,  -5.38617074e-02,
        -4.01346721e-02,

In [9]:
model.docvecs['TEST_POS_0']

array([ 0.06397799,  0.04854327,  0.04797371, -0.05795331,  0.00660171,
       -0.1534498 ,  0.03194107,  0.0228539 , -0.01616059, -0.10496912,
        0.09088878, -0.0369557 , -0.11332453, -0.02765095,  0.0504078 ,
        0.00884504, -0.01455043,  0.06211093,  0.03845689, -0.07290225,
       -0.00616732, -0.0376738 , -0.108957  , -0.08273569,  0.10881425,
        0.00491898,  0.05228799, -0.03005295,  0.09054932, -0.011918  ,
        0.03896611,  0.03833171,  0.07846707, -0.1127496 , -0.02024343,
        0.03830462, -0.00143523,  0.03447235, -0.02882409, -0.05632888,
       -0.06388983,  0.03593896,  0.10247084, -0.0128594 ,  0.01833743,
       -0.03917554, -0.04268269, -0.05542992, -0.04303888, -0.04696486,
       -0.00711204,  0.00519106, -0.03470116, -0.05322464,  0.02282947,
       -0.03598513,  0.00314935, -0.07143622,  0.06212395, -0.1100677 ,
        0.04408289, -0.05736115,  0.01436321, -0.03038301, -0.04009873,
        0.00391322,  0.03688042,  0.08665289,  0.11611758, -0.04

In [10]:
logger.info('Sentiment')
train_arrays = numpy.zeros((210+154, SIZE))
train_labels = numpy.zeros(210+154)

for i in range(210):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_labels[i] = 1
for i in range(154):
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[210 + i] = model.docvecs[prefix_train_neg]
    train_labels[210 + i] = 0

2017-04-10 02:31:33,112 : INFO : Sentiment


In [11]:
test_arrays = numpy.zeros((52+38, SIZE))
test_labels = numpy.zeros(52+38)

for i in range(52):
    prefix_test_pos = 'TEST_POS_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_labels[i] = 1
for i in range(38):
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[52 + i] = model.docvecs[prefix_test_neg]
    test_labels[52 + i] = 0


In [12]:
from sklearn.linear_model import LogisticRegression

logger.info('Fitting')
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

print(classifier.score(test_arrays, test_labels))


2017-04-10 02:31:33,296 : INFO : Fitting


0.6


In [13]:
from sklearn import svm
classifier = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
classifier.fit(train_arrays, train_labels)
print(classifier.score(test_arrays,test_labels))

0.577777777778


In [19]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout,Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, AveragePooling1D
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from keras.utils import np_utils
import numpy as np
batch_size = 20
num_classes = 2
epochs = 15

# input image dimensions
img_rows, img_cols = 28, 28

# # the data, shuffled and split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)
x_train=train_arrays.reshape(train_arrays.shape[0], SIZE,1)
x_test=test_arrays.reshape(test_arrays.shape[0], SIZE,1)
input_shape = (SIZE)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
y_train=np_utils.to_categorical(train_labels, 2)
y_test=np_utils.to_categorical(test_labels, 2)
#print(y_test)
num_classes=2
sequence_input = (SIZE,1)
Kmodel = Sequential()
Kmodel.add(Conv1D(64, kernel_size=1,activation='linear',input_shape=sequence_input))
Kmodel.add(LeakyReLU(alpha=.001))
#print(Kmodel.output_shape)
Kmodel.add(AveragePooling1D(3))
#Kmodel.add(Conv1D(128, 2, activation='linear'))
#Kmodel.add(LeakyReLU(alpha=.001)) 
#Kmodel.add(AveragePooling1D())
#print(Kmodel.output_shape)
Kmodel.add(Flatten())
#print(Kmodel.output_shape)
Kmodel.add(Dense(128, activation='linear'))
Kmodel.add(LeakyReLU(alpha=.001))
#print(Kmodel.output_shape)
Kmodel.add(Dense(num_classes, activation='softmax'))
#print(Kmodel.output_shape)
Kmodel.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
Kmodel.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1)
score = Kmodel.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
Kmodel.metrics_names

Epoch 1/15
364/364 [==============================] - 0s - loss: 0.6906 - acc: 0.5659     
Epoch 2/15
364/364 [==============================] - 0s - loss: 0.6864 - acc: 0.5769     
Epoch 3/15
364/364 [==============================] - 0s - loss: 0.6858 - acc: 0.5769     
Epoch 4/15
364/364 [==============================] - 0s - loss: 0.6843 - acc: 0.5769     
Epoch 5/15
364/364 [==============================] - 0s - loss: 0.6841 - acc: 0.5769     
Epoch 6/15
364/364 [==============================] - 0s - loss: 0.6849 - acc: 0.5769     
Epoch 7/15
364/364 [==============================] - 0s - loss: 0.6851 - acc: 0.5769     
Epoch 8/15
364/364 [==============================] - 0s - loss: 0.6833 - acc: 0.5769     
Epoch 9/15
364/364 [==============================] - 0s - loss: 0.6830 - acc: 0.5769     
Epoch 10/15
364/364 [==============================] - 0s - loss: 0.6832 - acc: 0.5769     
Epoch 11/15
364/364 [==============================] - 0s - loss: 0.6841 - acc: 0.5769   

['loss', 'acc']

import numpy as np
np.random.seed(123)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, normalization
from keras.utils import np_utils
from keras.datasets import mnist, cifar10
from keras.layers.advanced_activations import LeakyReLU
from sklearn.decomposition import PCA


# Basic Prepocess function for MNIST
# Uses PCA and Normalization
def pca_mnist(X, X_test):
    X = X.astype('float32') / 255.
    X_test = X_test.astype('float32') / 255.
    X = X.reshape((len(X), np.prod(X.shape[1:])))
    X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
    print(X.shape)
    pca = PCA(n_components=100)
    pca.fit(X)
    X = pca.inverse_transform(pca.transform(X))
    X_test = pca.inverse_transform(pca.transform(X_test))
    X = X.reshape(X.shape[0], 1, 28, 28)
    X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
    print(X.shape)
    return X, X_test

#Preprocess for cifar10
#Doesn't really affect performance
def pca_cifar_pre(X):
    X2 = np.transpose(X, (3, 0, 1, 2))
    temp1 = 0
    temp2 = 0
    temp3 = 0
    for i in range(3): #individual PCA for each color 
        temp_X = X2[i][:][:][:]
        temp_X = np.transpose(temp_X, (0, 2, 1))
        temp_X = pca_cifar(temp_X)
        if i == 0:
            temp1 = temp_X
        elif i == 1:
            temp2 = temp_X
        else:
            temp3 = temp_X
    X_temp = np.array([temp1, temp2, temp3])
    # print(X.shape)
    X = np.transpose(X, (1, 2, 3, 0))
    X = X.astype('float32') / 255.
    X = X.reshape(X.shape[0], 3, 32, 32)
    return X


def pca_cifar(X):
    X = X.astype('float32') / 255.
    X = X.reshape((len(X), np.prod(X.shape[1:])))
    print(X.shape)
    pca = PCA(n_components=1024)
    pca.fit(X)
    X = pca.inverse_transform(pca.transform(X))
    X = X.reshape(X.shape[0], 32, 32)
    print(X.shape)
    return X


(X_train, y_train), (X_test, y_test) = mnist.load_data()



# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# 7. Define model architecture
model = Sequential()

model.add(Convolution(64, kernel_size=(5, 5), data_format="channels_first", activation='relu',input_shape=(1, 28, 28)))


print(model.output_shape)
model.add(Convolution2D(64,kernel_size=(5, 5), activation='linear')) #Convolutinal Leaky Layer
model.add(LeakyReLU(alpha=.001))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2, 2))) #Vanila Pooling
print(model.output_shape)
model.add(Dropout(0.25))

model.add(Flatten())
print(model.output_shape)
model.add(Dense(128, activation='relu'))
print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


model.summary

#Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
#Fit model on training data
model.fit(X_train, Y_train, batch_size=32, epochs=20, verbose=1)


#Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=1)
print(score)
